iMPORTS

In [48]:
import json
import requests
from bs4 import BeautifulSoup

In [20]:
with open("./cookie.json","r",encoding="UTF-8") as jf:
    headers = json.load(jf)

1. send a request for accessing first webpage with opinions about products


2. If responce is OK, parse HTML page content into DOM structure 

3. Extract from DOM structure  opinions and their components

4. Assign opinions with their cmponents to complex data structures


5. If there are more pages with opinions, repeat all steps 1-5

In [57]:
try:
    url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination_next")["href"]
except TypeError:
    url = None

In [64]:
url = "https://www.ceneo.pl/169771365#tab=reviews"
all_opinions = []
while url:
    response = requests.get(url, headers = headers)
    if response.status_code == 200:
     page_dom = BeautifulSoup(response.text, 'html.parser')
     opinions = page_dom.select("div.js_product-review:not(.user-post--highlights)")
   
     for opinion in opinions:
        single_opinion = {}
        single_opinion['opinion_id'] = opinion["data-entry-id"]
        single_opinion['author'] = opinion.select_one("span.user-post__author-name").text.strip()
        try:
            single_opinion['recommend'] = opinion.select_one("span.user-post__author-recomendation> em").text.strip()
        except AttributeError:
            single_opinion['recommend'] = None
        single_opinion['stars'] = opinion.select_one("span.user-post__score-count").text.strip()
        single_opinion['content'] = opinion.select_one("div.user-post__text").text.strip()
        single_opinion['pros'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--positive")]
        single_opinion['cons'] = [tag.text.strip() for tag in opinion.select("div.review-feature__item--negative")]
        single_opinion['up_votes'] = opinion.select_one("button.vote-yes")["data-total-vote"].strip()
        single_opinion['down_votes'] = opinion.select_one("button.vote-no")["data-total-vote"].strip()
        single_opinion['published'] = opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip()
        try:
            single_opinion['purshaped'] = opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip()
        except TypeError:
            single_opinion['purshaped'] = None
        all_opinions.append(single_opinion)
        print(single_opinions)
        try:
          url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination_next")["href"]
        except TypeError:
          url = None

[{'opinion_id': '19175428', 'author': 'r...2', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Wspaniałe urządzenie spełnia wszystkie moje oczekiwania', 'pros': ['jakość wykonania', 'łatwość czyszczenia', 'pojemność', 'szybkość nagrzewania'], 'cons': [], 'up_votes': '2', 'down_votes': '0', 'published': '2024-11-01 10:42:48', 'purshaped': '2024-10-23 21:35:35'}]
[{'opinion_id': '19175428', 'author': 'r...2', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Wspaniałe urządzenie spełnia wszystkie moje oczekiwania', 'pros': ['jakość wykonania', 'łatwość czyszczenia', 'pojemność', 'szybkość nagrzewania'], 'cons': [], 'up_votes': '2', 'down_votes': '0', 'published': '2024-11-01 10:42:48', 'purshaped': '2024-10-23 21:35:35'}, {'opinion_id': '19524088', 'author': 'o...a', 'recommend': 'Polecam', 'stars': '5/5', 'content': 'Wszystko zgodnie z zamówieniem', 'pros': ['efektywność', 'jakość wykonania', 'komfort użytkowania', 'łatwość czyszczenia', 'pojemność', 'szybkość nagrzewania'], 'cons